In [9]:
%%writefile openmp_stats.cpp
#include <iostream>
#include <vector>
#include <omp.h>

using namespace std;

int main() {
    const int N = 10'000'000;
    vector<double> data(N);

    // Последовательная часть: инициализация массива
    double t_start = omp_get_wtime();
    for (int i = 0; i < N; i++) {
        data[i] = 1.0;
    }
    double t_init = omp_get_wtime();

    double sum = 0.0;
    double sumSquares = 0.0;

    // Параллельная часть
    double t_parallel_start = omp_get_wtime();

    #pragma omp parallel for reduction(+:sum, sumSquares)
    for (int i = 0; i < N; i++) {
        sum += data[i];
        sumSquares += data[i] * data[i];
    }

    double t_parallel_end = omp_get_wtime();

    // Последовательная часть: финальные вычисления
    double mean = sum / N;
    double variance = sumSquares / N - mean * mean;
    double t_end = omp_get_wtime();

    cout << "Сумма: " << sum << endl;
    cout << "Среднее: " << mean << endl;
    cout << "Дисперсия: " << variance << endl;

    cout << "\nВремя инициализации (последовательно): "
         << (t_init - t_start) << " сек" << endl;

    cout << "Время параллельных вычислений: "
         << (t_parallel_end - t_parallel_start) << " сек" << endl;

    cout << "Общее время выполнения: "
         << (t_end - t_start) << " сек" << endl;

    return 0;
}


Overwriting openmp_stats.cpp


In [10]:
!g++ -fopenmp -O2 openmp_stats.cpp -o openmp_stats


In [11]:
!./openmp_stats


Сумма: 1e+07
Среднее: 1
Дисперсия: 0

Время инициализации (последовательно): 0.00864899 сек
Время параллельных вычислений: 0.0145394 сек
Общее время выполнения: 0.0231888 сек


In [12]:
%%writefile cuda_memory_access.cu
#include <iostream>
#include <cuda_runtime.h>

using namespace std;

const int N = 1 << 20;        // ~1 000 000 элементов
const int BLOCK_SIZE = 256;

// ----------------------------
// 1. Коалесцированный доступ
// ----------------------------
__global__ void coalescedKernel(float* data) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) {
        data[idx] *= 2.0f;
    }
}

// --------------------------------
// 2. Некоалесцированный доступ
// --------------------------------
__global__ void nonCoalescedKernel(float* data) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    if (idx < N) {
        int badIndex = (idx * 32) % N;
        data[badIndex] *= 2.0f;
    }
}

// ------------------------------------
// 3. Коалесцированный + shared memory
// ------------------------------------
__global__ void sharedMemoryKernel(float* data) {
    __shared__ float buffer[BLOCK_SIZE];

    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int tid = threadIdx.x;

    if (idx < N) {
        buffer[tid] = data[idx];
    }
    __syncthreads();

    if (idx < N) {
        buffer[tid] *= 2.0f;
    }
    __syncthreads();

    if (idx < N) {
        data[idx] = buffer[tid];
    }
}

// ----------------------------
// Замер времени
// ----------------------------
float measureKernel(void (*kernel)(float*), float* d_data, dim3 grid, dim3 block) {
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    kernel<<<grid, block>>>(d_data);
    cudaEventRecord(stop);

    cudaEventSynchronize(stop);
    float time;
    cudaEventElapsedTime(&time, start, stop);

    return time;
}

int main() {
    float* h_data = new float[N];
    for (int i = 0; i < N; i++) {
        h_data[i] = 1.0f;
    }

    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);

    dim3 block(BLOCK_SIZE);
    dim3 grid((N + BLOCK_SIZE - 1) / BLOCK_SIZE);

    float t1 = measureKernel(coalescedKernel, d_data, grid, block);
    float t2 = measureKernel(nonCoalescedKernel, d_data, grid, block);
    float t3 = measureKernel(sharedMemoryKernel, d_data, grid, block);

    cout << "Коалесцированный доступ: " << t1 << " мс" << endl;
    cout << "Некoалесцированный доступ: " << t2 << " мс" << endl;
    cout << "Shared memory: " << t3 << " мс" << endl;

    cudaFree(d_data);
    delete[] h_data;

    return 0;
}


Writing cuda_memory_access.cu


In [13]:
!nvcc -arch=sm_75 cuda_memory_access.cu -o mem_access
!./mem_access


cuda_memory_access.cu(24): warning #177-D: variable "stride" was declared but never referenced
      int stride = blockDim.x * gridDim.x;
          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

Коалесцированный доступ: 0.16256 мс
Некoалесцированный доступ: 0.284832 мс
Shared memory: 0.046016 мс


In [14]:
%%writefile hybrid_profiling.cu
#include <iostream>
#include <cuda_runtime.h>
#include <omp.h>

using namespace std;

const int N = 1 << 20;       // ~1 000 000 элементов
const int HALF = N / 2;
const int BLOCK_SIZE = 256;

// CUDA-ядро
__global__ void gpuKernel(float* data, int offset, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[offset + idx] *= 2.0f;
    }
}

int main() {
    float* h_data;
    cudaMallocHost(&h_data, N * sizeof(float)); // pinned memory

    for (int i = 0; i < N; i++) {
        h_data[i] = 1.0f;
    }

    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));

    cudaStream_t stream;
    cudaStreamCreate(&stream);

    double startTotal = omp_get_wtime();

    // Асинхронная передача CPU → GPU
    double t_copy_start = omp_get_wtime();
    cudaMemcpyAsync(d_data + HALF,
                    h_data + HALF,
                    HALF * sizeof(float),
                    cudaMemcpyHostToDevice,
                    stream);
    double t_copy_end = omp_get_wtime();

    // Параллельная обработка CPU
    double t_cpu_start = omp_get_wtime();
    #pragma omp parallel for
    for (int i = 0; i < HALF; i++) {
        h_data[i] *= 2.0f;
    }
    double t_cpu_end = omp_get_wtime();

    // Запуск CUDA-ядра
    int gridSize = (HALF + BLOCK_SIZE - 1) / BLOCK_SIZE;
    double t_gpu_start = omp_get_wtime();
    gpuKernel<<<gridSize, BLOCK_SIZE, 0, stream>>>(d_data, HALF, HALF);
    cudaStreamSynchronize(stream);
    double t_gpu_end = omp_get_wtime();

    // Асинхронная передача GPU → CPU
    cudaMemcpyAsync(h_data + HALF,
                    d_data + HALF,
                    HALF * sizeof(float),
                    cudaMemcpyDeviceToHost,
                    stream);
    cudaStreamSynchronize(stream);

    double endTotal = omp_get_wtime();

    cout << "Время передачи CPU → GPU: "
         << (t_copy_end - t_copy_start) << " сек" << endl;

    cout << "Время CPU вычислений: "
         << (t_cpu_end - t_cpu_start) << " сек" << endl;

    cout << "Время GPU вычислений: "
         << (t_gpu_end - t_gpu_start) << " сек" << endl;

    cout << "Общее время выполнения: "
         << (endTotal - startTotal) << " сек" << endl;

    cudaFree(d_data);
    cudaFreeHost(h_data);
    cudaStreamDestroy(stream);

    return 0;
}


Writing hybrid_profiling.cu


In [15]:
!nvcc -arch=sm_75 -Xcompiler -fopenmp hybrid_profiling.cu -o hybrid_prof
!./hybrid_prof


Время передачи CPU → GPU: 2.0473e-05 сек
Время CPU вычислений: 0.00130168 сек
Время GPU вычислений: 0.000162436 сек
Общее время выполнения: 0.00167191 сек


In [16]:
%%writefile mpi_scaling.cpp
#include <mpi.h>
#include <iostream>
#include <vector>
#include <cstdlib>
#include <limits>

using namespace std;

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Размер массива для strong scaling
    const int N = 1'000'000;

    int localN = N / size;

    vector<double> localData(localN);

    // Инициализация локальных данных
    for (int i = 0; i < localN; i++) {
        localData[i] = 1.0;
    }

    double t_start = MPI_Wtime();

    // Локальные агрегаты
    double localSum = 0.0;
    double localMin = numeric_limits<double>::max();
    double localMax = numeric_limits<double>::lowest();

    for (double x : localData) {
        localSum += x;
        localMin = min(localMin, x);
        localMax = max(localMax, x);
    }

    // Глобальные агрегаты
    double globalSum, globalMin, globalMax;

    MPI_Reduce(&localSum, &globalSum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);
    MPI_Reduce(&localMin, &globalMin, 1, MPI_DOUBLE, MPI_MIN, 0, MPI_COMM_WORLD);
    MPI_Reduce(&localMax, &globalMax, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD);

    double t_end = MPI_Wtime();

    if (rank == 0) {
        cout << "Процессов: " << size << endl;
        cout << "Сумма: " << globalSum << endl;
        cout << "Мин: " << globalMin << endl;
        cout << "Макс: " << globalMax << endl;
        cout << "Время выполнения: "
             << (t_end - t_start) << " сек\n" << endl;
    }

    MPI_Finalize();
    return 0;
}


Writing mpi_scaling.cpp


In [17]:
!mpic++ mpi_scaling.cpp -O2 -o mpi_scaling


In [18]:
!mpirun --allow-run-as-root --oversubscribe -np 1 ./mpi_scaling
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_scaling
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_scaling
!mpirun --allow-run-as-root --oversubscribe -np 8 ./mpi_scaling


Процессов: 1
Сумма: 1e+06
Мин: 1
Макс: 1
Время выполнения: 0.00161841 сек

Процессов: 2
Сумма: 1e+06
Мин: 1
Макс: 1
Время выполнения: 0.00077528 сек

Процессов: 4
Сумма: 1e+06
Мин: 1
Макс: 1
Время выполнения: 0.00105961 сек

Процессов: 8
Сумма: 1e+06
Мин: 1
Макс: 1
Время выполнения: 0.00553872 сек

